# 04 Walk-Forward Validation

Stability of recession prediction across time.


## Table of Contents
- Walk-forward splits
- Metric stability
- Failure analysis


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Matching Guide
- `docs/guides/03_classification/04_walk_forward_validation.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/03_classification/04_walk_forward_validation.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Walk-forward evaluation: measure stability across time.


## Your Turn: Implement walk-forward


In [ ]:
from src import evaluation

# TODO: Use evaluation.walk_forward_splits to evaluate across folds
# Plot metrics over time and identify unstable periods
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Walk-forward splits</summary>

```python
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from src import evaluation

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True).dropna()
target = 'target_recession_next_q'
drop_cols = {target, 'recession', 'GDPC1', 'gdp_growth_qoq', 'gdp_growth_qoq_annualized', 'gdp_growth_yoy'}
X = df[[c for c in df.columns if c not in drop_cols]].astype(float)
y = df[target].astype(int).to_numpy()

clf = Pipeline([('scaler', StandardScaler()), ('m', LogisticRegression(max_iter=5000))])

metrics = []
for split in evaluation.walk_forward_splits(len(df), initial_train_size=20, test_size=4, step_size=2):
    clf.fit(X.iloc[split.train_slice], y[split.train_slice])
    p = clf.predict_proba(X.iloc[split.test_slice])[:,1]
    m = evaluation.classification_metrics(y[split.test_slice], p)
    metrics.append(m)

pd.DataFrame(metrics).head()
```

</details>

<details><summary>Solution: Metric stability</summary>

```python
# Plot ROC-AUC/PR-AUC over folds and inspect variance.
```

</details>

<details><summary>Solution: Failure analysis</summary>

```python
# Identify folds with worst metrics and map to dates.
```

</details>

